In [1]:
import numpy as np
import gym
import gym_tictactoe as t3
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from os import path
from sklearn.preprocessing import LabelBinarizer
from timeit import Timer
timer = Timer()

In [2]:
DEBUG = False

n_input = (3 * 3 * 3) * 3
n_hidden = 100
n_output = 3 * 3 * 3
learning_rate = 0.01

checkpoint_path = './my_tictactoe_h{}_lr{}.ckpt'.format(n_hidden, learning_rate)
initializer = tf.contrib.layers.variance_scaling_initializer()

env = gym.make('tictactoe-v0')

In [3]:
encoder = LabelBinarizer()
encoder.fit(np.array([[0], [1], [2]]))

def convert_game_to_x_state(obs):
    # gym_tictactoe now supports int-encoded world
    world = np.array(obs, dtype=np.float32)
    data = list(map(lambda x: [x], world.flatten()))
    return encoder.transform(data).flatten()

def convert_action_to_step(action, player):
    action = int(action)
    val = 0
    multiplier = 1
    while action:
        val += (action%3)*multiplier
        multiplier *= 10
        action //= 3
    
    return str(player) + str(val).zfill(3)

def discount_rewards(rewards, discount_rate):
    reward_epsilon = 0.1
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = (rewards[step] + reward_epsilon) + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate=discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

In [4]:
player_scopes = ['player_1', 'player_2']
all_logits = []
outputs = []
ys = []
cross_entropies = []
training_ops = []
all_gradient_placeholders = []
all_gradients = []
optimizers = []

X_state = tf.placeholder(shape=(None, n_input), dtype=tf.float32, name='X')
global_step = tf.Variable(0, trainable=False, name='global_step')

for scope in player_scopes:
    with tf.variable_scope(scope):
        hidden = fully_connected(X_state, n_hidden, activation_fn=tf.nn.relu, weights_initializer=initializer)
        logits = fully_connected(hidden, n_output, activation_fn=None, weights_initializer=initializer)
        all_logits.append(logits)

        output = tf.contrib.layers.softmax(logits)
        outputs.append(output)

        y = tf.to_float(tf.multinomial(tf.log(output), num_samples=n_output))
        ys.append(y)

        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
        cross_entropies.append(cross_entropy)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        optimizers.append(optimizer)

        trainable_variables = tf.trainable_variables(scope=scope)
        grads_and_vars = optimizer.compute_gradients(cross_entropy, trainable_variables)
        gradients = [grad for grad, variable in grads_and_vars]
        all_gradients.append(gradients)
        gradient_placeholders = []
        grads_and_vars_feed = []

        for grad, variable in grads_and_vars:
            gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
            gradient_placeholders.append(gradient_placeholder)
            grads_and_vars_feed.append((gradient_placeholder, variable))
        training_op = optimizer.apply_gradients(grads_and_vars_feed, global_step=global_step)
        training_ops.append(training_op)
        all_gradient_placeholders.append(gradient_placeholders)

        
init = tf.global_variables_initializer()
saver = tf.train.Saver()

file_writer = tf.summary.FileWriter('logs', tf.get_default_graph())
file_writer.close()

In [5]:
n_games = 10
n_rounds = 0 #### DON'T PLAY!

with tf.Session() as sess:
    init.run()
    for r in range(n_rounds):
        gen = r % len(player_scopes)
        adv = (r+1) % 2
        round_rewards = []
        round_gradients = []
        start = timer.timer()
        for i in range(n_games):
            env = gym.make('tictactoe-v0')
            obs = env.reset()
            turn = 0
            done = False
            game_rewards = 0
            while not done:
                player = (turn%2) + 1
                world = convert_game_to_x_state(obs)            
                with tf.variable_scope(player_scopes[gen]):
                    action, grad_result = sess.run([ys[gen], all_gradients[gen]], feed_dict={X_state: [world]})
                action_taken = int(action[0][0])
                step = convert_action_to_step(action_taken, player)
                obs, reward, done, info = env.step(step)
                print(action_taken, end=',')
                game_rewards += reward
                turn += 1
                if done:
                    break
            print(' =', game_rewards)
#             game_gradients = gradients[gen].eval(feed_dict=None)
            round_rewards.append(game_rewards)
            round_gradients.append(grad_result)
        print('Game rewards ({}):{}'.format((timer.timer() - start)//1, game_rewards))

In [8]:
def play_game(env, gen, adv, gen_starts=True):
    # Gen starts the game as player 1
    gen_player = 1 if gen_starts else 2
    adv_player = 2 if gen_starts else 1
    obs = env.reset()
    done = False
    action_rewards = []
    action_gradients = []
    while not done:
        if gen_starts:
            world = convert_game_to_x_state(obs)
            with tf.variable_scope(player_scopes[gen]):
                gen_action, gen_grad_result = sess.run([ys[gen], all_gradients[gen]], feed_dict={X_state: [world]})
            gen_action_taken = int(gen_action[0][0])
            gen_step = convert_action_to_step(gen_action_taken, gen_player)
            obs, gen_reward, done, info = env.step(gen_step)
            if DEBUG:
                print('G({})'.format(gen_step), end=' ')

            action_rewards.append(gen_reward)
            action_gradients.append(gen_grad_result)

            if done:
                break

            world = convert_game_to_x_state(obs)
            with tf.variable_scope(player_scopes[adv]):
                adv_action = sess.run(ys[adv], feed_dict={X_state: [world]})
            adv_action_taken = int(adv_action[0][0])
            adv_step = convert_action_to_step(adv_action_taken, adv_player)
            obs, adv_reward, done, info = env.step(adv_step)

            if DEBUG:
                print('A({})'.format(adv_step), end=' ')

        else:
            world = convert_game_to_x_state(obs)
            with tf.variable_scope(player_scopes[adv]):
                adv_action = sess.run(ys[adv], feed_dict={X_state: [world]})
            adv_action_taken = int(adv_action[0][0])
            adv_step = convert_action_to_step(adv_action_taken, adv_player)
            obs, adv_reward, done, info = env.step(adv_step)

            if DEBUG:
                print('A({})'.format(adv_step), end=' ')

            if done:
                break

            world = convert_game_to_x_state(obs)
            with tf.variable_scope(player_scopes[gen]):
                gen_action, gen_grad_result = sess.run([ys[gen], all_gradients[gen]], feed_dict={X_state: [world]})
            gen_action_taken = int(gen_action[0][0])
            gen_step = convert_action_to_step(gen_action_taken, gen_player)
            obs, gen_reward, done, info = env.step(gen_step)
            if DEBUG:
                print('G({})'.format(gen_step), end=' ')

            action_rewards.append(gen_reward)
            action_gradients.append(gen_grad_result)
            
    return action_rewards, action_gradients

def play_with_human(env):
    obs = env.reset()
    done = False
    player = 0
    with tf.Session() as sess:
        saver.restore(sess, checkpoint_path)
        with tf.variable_scope('player_1'):
            while not done:
                env.render()
                step = input()
                obs, reward, done, info = env.step('{}{}'.format(player%2+1, step))

                if done:
                    break

                player += 1
                world = convert_game_to_x_state(obs)
                action_result = sess.run(ys[0], feed_dict={X_state: [world]})
                action = int(action_result[0][0])
                step = convert_action_to_step(action, player%2+1)
                obs, reward, done, info = env.step(step)

                player += 1
    env.render()

In [18]:
n_games_per_iteration = 50
n_iterations = 10000
discount_rate = 0.95

with tf.Session() as sess:
    if path.exists(checkpoint_path + '.meta'):
        saver.restore(sess, checkpoint_path)
    else: 
        init.run()
        
    print('GStep at start:', global_step.eval())
    
    for it in range(n_iterations):
        gen = (0 + it) % 2
        adv = (1 + it) % 2

        iteration_rewards = []
        iteration_gradients = []
        
        game_play_start = timer.timer()
        for g in range(n_games_per_iteration):
            game_rewards, game_gradients = play_game(env, gen, adv, False)
            if DEBUG:
                print('rewards:', game_rewards)

            iteration_rewards.append(game_rewards)
            iteration_gradients.append(game_gradients)
            if g%5 == 0:
                print('.', end='')
            
        for g in range(n_games_per_iteration):
            game_rewards, game_gradients = play_game(env, gen, adv, True)
            if DEBUG:
                print('rewards:', game_rewards)

            iteration_rewards.append(game_rewards)
            iteration_gradients.append(game_gradients)
            if g%5 == 0:
                print('.', end='')
        game_play_end = timer.timer()
        
        training_start = timer.timer()
        feed_dict = {}
        iteration_rewards = discount_and_normalize_rewards(iteration_rewards, discount_rate)
        for var_index, grad_placeholder in enumerate(all_gradient_placeholders[gen]):
            mean_gradients = np.mean(
                [ reward * iteration_gradients[game_index][step][var_index]
                     for game_index, rewards in enumerate(iteration_rewards)
                     for step, reward in enumerate(rewards)],
                axis=0)
            feed_dict[grad_placeholder] = mean_gradients
    
        training_end = timer.timer()
        
        print('Step:{}'.format(global_step.eval()),
              'Gameplay:{}s'.format((10*(game_play_end-game_play_start)//1)/10),
              'Training:{}s'.format((10*(training_end-training_start)//1)/10))

        with tf.variable_scope(player_scopes[0]):
            sess.run(training_ops[gen], feed_dict=feed_dict)
        saver.save(sess, checkpoint_path)

INFO:tensorflow:Restoring parameters from ./my_tictactoe_h100_lr0.01.ckpt
GStep at start: 10125
....................Step:10125 Gameplay:0.3s Training:0.0s
....................Step:10126 Gameplay:0.3s Training:0.0s
....................Step:10127 Gameplay:0.2s Training:0.0s
....................Step:10128 Gameplay:0.2s Training:0.0s
....................Step:10129 Gameplay:0.3s Training:0.0s
....................Step:10130 Gameplay:0.2s Training:0.0s
....................Step:10131 Gameplay:0.3s Training:0.0s
....................Step:10132 Gameplay:0.3s Training:0.0s
....................Step:10133 Gameplay:0.2s Training:0.0s
....................Step:10134 Gameplay:0.2s Training:0.0s
....................Step:10135 Gameplay:0.2s Training:0.0s
....................Step:10136 Gameplay:0.2s Training:0.0s
....................Step:10137 Gameplay:0.2s Training:0.0s
....................Step:10138 Gameplay:0.2s Training:0.0s
....................Step:10139 Gameplay:0.2s Training:0.0s
...................

....................Step:10263 Gameplay:0.2s Training:0.0s
....................Step:10264 Gameplay:0.3s Training:0.0s
....................Step:10265 Gameplay:0.2s Training:0.0s
....................Step:10266 Gameplay:0.2s Training:0.0s
....................Step:10267 Gameplay:0.2s Training:0.0s
....................Step:10268 Gameplay:0.2s Training:0.0s
....................Step:10269 Gameplay:0.2s Training:0.0s
....................Step:10270 Gameplay:0.2s Training:0.0s
....................Step:10271 Gameplay:0.2s Training:0.0s
....................Step:10272 Gameplay:0.2s Training:0.0s
....................Step:10273 Gameplay:0.2s Training:0.0s
....................Step:10274 Gameplay:0.2s Training:0.0s
....................Step:10275 Gameplay:0.2s Training:0.0s
....................Step:10276 Gameplay:0.2s Training:0.0s
....................Step:10277 Gameplay:0.2s Training:0.0s
....................Step:10278 Gameplay:0.2s Training:0.0s
....................Step:10279 Gameplay:0.2s Training:0.

....................Step:10402 Gameplay:0.2s Training:0.0s
....................Step:10403 Gameplay:0.2s Training:0.0s
....................Step:10404 Gameplay:0.2s Training:0.0s
....................Step:10405 Gameplay:0.2s Training:0.0s
....................Step:10406 Gameplay:0.2s Training:0.0s
....................Step:10407 Gameplay:0.2s Training:0.0s
....................Step:10408 Gameplay:0.2s Training:0.0s
....................Step:10409 Gameplay:0.2s Training:0.0s
....................Step:10410 Gameplay:0.2s Training:0.0s
....................Step:10411 Gameplay:0.2s Training:0.0s
....................Step:10412 Gameplay:0.2s Training:0.0s
....................Step:10413 Gameplay:0.2s Training:0.0s
....................Step:10414 Gameplay:0.2s Training:0.0s
....................Step:10415 Gameplay:0.2s Training:0.0s
....................Step:10416 Gameplay:0.2s Training:0.0s
....................Step:10417 Gameplay:0.2s Training:0.0s
....................Step:10418 Gameplay:0.2s Training:0.

....................Step:10541 Gameplay:0.2s Training:0.0s
....................Step:10542 Gameplay:0.2s Training:0.0s
....................Step:10543 Gameplay:0.2s Training:0.0s
....................Step:10544 Gameplay:0.2s Training:0.0s
....................Step:10545 Gameplay:0.2s Training:0.0s
....................Step:10546 Gameplay:0.2s Training:0.0s
....................Step:10547 Gameplay:0.2s Training:0.0s
....................Step:10548 Gameplay:0.2s Training:0.0s
....................Step:10549 Gameplay:0.2s Training:0.0s
....................Step:10550 Gameplay:0.2s Training:0.0s
....................Step:10551 Gameplay:0.2s Training:0.0s
....................Step:10552 Gameplay:0.2s Training:0.0s
....................Step:10553 Gameplay:0.2s Training:0.0s
....................Step:10554 Gameplay:0.2s Training:0.0s
....................Step:10555 Gameplay:0.2s Training:0.0s
....................Step:10556 Gameplay:0.2s Training:0.0s
....................Step:10557 Gameplay:0.2s Training:0.

....................Step:10680 Gameplay:0.2s Training:0.0s
....................Step:10681 Gameplay:0.2s Training:0.0s
....................Step:10682 Gameplay:0.2s Training:0.0s
....................Step:10683 Gameplay:0.2s Training:0.0s
....................Step:10684 Gameplay:0.2s Training:0.0s
....................Step:10685 Gameplay:0.2s Training:0.0s
....................Step:10686 Gameplay:0.2s Training:0.0s
....................Step:10687 Gameplay:0.2s Training:0.0s
....................Step:10688 Gameplay:0.2s Training:0.0s
....................Step:10689 Gameplay:0.2s Training:0.0s
....................Step:10690 Gameplay:0.2s Training:0.0s
....................Step:10691 Gameplay:0.2s Training:0.0s
....................Step:10692 Gameplay:0.2s Training:0.0s
....................Step:10693 Gameplay:0.2s Training:0.0s
....................Step:10694 Gameplay:0.2s Training:0.0s
....................Step:10695 Gameplay:0.2s Training:0.0s
....................Step:10696 Gameplay:0.2s Training:0.

...

....................Step:19437 Gameplay:0.2s Training:0.0s
....................Step:19438 Gameplay:0.2s Training:0.0s
....................Step:19439 Gameplay:0.2s Training:0.0s
....................Step:19440 Gameplay:0.2s Training:0.0s
....................Step:19441 Gameplay:0.2s Training:0.0s
....................Step:19442 Gameplay:0.2s Training:0.0s
....................Step:19443 Gameplay:0.2s Training:0.0s
....................Step:19444 Gameplay:0.2s Training:0.0s
....................Step:19445 Gameplay:0.2s Training:0.0s
....................Step:19446 Gameplay:0.2s Training:0.0s
....................Step:19447 Gameplay:0.2s Training:0.0s
....................Step:19448 Gameplay:0.2s Training:0.0s
....................Step:19449 Gameplay:0.2s Training:0.0s
....................Step:19450 Gameplay:0.2s Training:0.0s
....................Step:19451 Gameplay:0.2s Training:0.0s
....................Step:19452 Gameplay:0.2s Training:0.0s
....................Step:19453 Gameplay:0.2s Training:0.

....................Step:19576 Gameplay:0.2s Training:0.0s
....................Step:19577 Gameplay:0.2s Training:0.0s
....................Step:19578 Gameplay:0.2s Training:0.0s
....................Step:19579 Gameplay:0.2s Training:0.0s
....................Step:19580 Gameplay:0.2s Training:0.0s
....................Step:19581 Gameplay:0.2s Training:0.0s
....................Step:19582 Gameplay:0.2s Training:0.0s
....................Step:19583 Gameplay:0.2s Training:0.0s
....................Step:19584 Gameplay:0.2s Training:0.0s
....................Step:19585 Gameplay:0.2s Training:0.0s
....................Step:19586 Gameplay:0.2s Training:0.0s
....................Step:19587 Gameplay:0.2s Training:0.0s
....................Step:19588 Gameplay:0.3s Training:0.0s
....................Step:19589 Gameplay:0.2s Training:0.0s
....................Step:19590 Gameplay:0.2s Training:0.0s
....................Step:19591 Gameplay:0.2s Training:0.0s
....................Step:19592 Gameplay:0.2s Training:0.

....................Step:19715 Gameplay:0.2s Training:0.0s
....................Step:19716 Gameplay:0.2s Training:0.0s
....................Step:19717 Gameplay:0.2s Training:0.0s
....................Step:19718 Gameplay:0.2s Training:0.0s
....................Step:19719 Gameplay:0.2s Training:0.0s
....................Step:19720 Gameplay:0.2s Training:0.0s
....................Step:19721 Gameplay:0.2s Training:0.0s
....................Step:19722 Gameplay:0.2s Training:0.0s
....................Step:19723 Gameplay:0.2s Training:0.0s
....................Step:19724 Gameplay:0.2s Training:0.0s
....................Step:19725 Gameplay:0.2s Training:0.0s
....................Step:19726 Gameplay:0.2s Training:0.0s
....................Step:19727 Gameplay:0.2s Training:0.0s
....................Step:19728 Gameplay:0.2s Training:0.0s
....................Step:19729 Gameplay:0.2s Training:0.0s
....................Step:19730 Gameplay:0.2s Training:0.0s
....................Step:19731 Gameplay:0.2s Training:0.

....................Step:19854 Gameplay:0.2s Training:0.0s
....................Step:19855 Gameplay:0.2s Training:0.0s
....................Step:19856 Gameplay:0.2s Training:0.0s
....................Step:19857 Gameplay:0.2s Training:0.0s
....................Step:19858 Gameplay:0.2s Training:0.0s
....................Step:19859 Gameplay:0.2s Training:0.0s
....................Step:19860 Gameplay:0.2s Training:0.0s
....................Step:19861 Gameplay:0.2s Training:0.0s
....................Step:19862 Gameplay:0.3s Training:0.0s
....................Step:19863 Gameplay:0.2s Training:0.0s
....................Step:19864 Gameplay:0.2s Training:0.0s
....................Step:19865 Gameplay:0.2s Training:0.0s
....................Step:19866 Gameplay:0.2s Training:0.0s
....................Step:19867 Gameplay:0.2s Training:0.0s
....................Step:19868 Gameplay:0.2s Training:0.0s
....................Step:19869 Gameplay:0.2s Training:0.0s
....................Step:19870 Gameplay:0.2s Training:0.

....................Step:19993 Gameplay:0.2s Training:0.0s
....................Step:19994 Gameplay:0.2s Training:0.0s
....................Step:19995 Gameplay:0.2s Training:0.0s
....................Step:19996 Gameplay:0.2s Training:0.0s
....................Step:19997 Gameplay:0.2s Training:0.0s
....................Step:19998 Gameplay:0.2s Training:0.0s
....................Step:19999 Gameplay:0.2s Training:0.0s
....................Step:20000 Gameplay:0.2s Training:0.0s
....................Step:20001 Gameplay:0.2s Training:0.0s
....................Step:20002 Gameplay:0.2s Training:0.0s
....................Step:20003 Gameplay:0.2s Training:0.0s
....................Step:20004 Gameplay:0.2s Training:0.0s
....................Step:20005 Gameplay:0.2s Training:0.0s
....................Step:20006 Gameplay:0.2s Training:0.0s
....................Step:20007 Gameplay:0.2s Training:0.0s
....................Step:20008 Gameplay:0.2s Training:0.0s
....................Step:20009 Gameplay:0.2s Training:0.

In [19]:
play_with_human(env)

INFO:tensorflow:Restoring parameters from ./my_tictactoe_h100_lr0.01.ckpt
- - -    - - -    - - -    
- - -    - - -    - - -    
- - -    - - -    - - -    
000
x - -    - - -    - - -    
o - -    - - -    - - -    
- - -    - - -    - - -    
011
x - -    - - -    - - -    
o x -    - - -    - - -    
- - -    - - -    - - -    


In [83]:
env._done

True